"flowchart" per l'applicazione.

1. lettura e preprocessing dei dati
2. lavorazione sul modello
3. generazione output 
4. valutazione output

In [1]:
# packages
import pandas as pd
import numpy as np

from claspy.segmentation import BinaryClaSPSegmentation

import ruptures as rpt

import stumpy
from aeon.segmentation import find_dominant_window_sizes

from aeon.segmentation import GreedyGaussianSegmenter

from aeon.segmentation import InformationGainSegmenter

from aeon.anomaly_detection import STRAY

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, make_scorer,mean_squared_error
import matplotlib.pyplot as plt
from aeon.visualisation import plot_series_with_change_points, plot_series_with_profiles
import seaborn as sns

sns.set_theme()
sns.set_color_codes()

C:\Users\tikyn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\aeon\base\__init__.py:24: FutureWarning: The aeon package will soon be releasing v1.0.0 with the removal of legacy modules and interfaces such as BaseTransformer and BaseForecaster. This will contain breaking changes. See aeon-toolkit.org for more information. Set aeon.AEON_DEPRECATION_WARNING or the AEON_DEPRECATION_WARNING environmental variable to 'False' to disable this warning.
  warnings.warn(


Extracted features. Use the index of this list to use with iloc[]
<ol start="0">
  <li>Kinetic Global</li>
  <li>Kinetic Chest</li>
  <li>Directness Head</li>
  <li>Density</li>
  <li>left wrist ke</li>
  <li>right wrist ke</li>
  <li>left ankle ke</li>
  <li>right ankle ke</li>
  <li>head ke</li>
  <li>crouch density</li>
  <li>left leg density</li>
  <li>right leg density</li>
  <li>left hand density</li>
  <li>right hand density</li>
  <li>hand density</li>
  <li>arto inferiore</li>
  <li>gamba (angolo tra le gambe)</li> 
  <li>coscia (angolo tra le due coscie)</li>
  <li>coscia dx (angolo tra coscia e colonna vertebrale)</li>
  <li>coscia sx</li>
  <li>gamba sx</li>
  <li>gamba dx</li>
  <li>braccio sx</li>
  <li>braccio dx</li>
  <li>avambraccio sx</li>
  <li>avambraccio dx</li>
  <li>ARIEL speed magnitude</li>
  <li>ARIEL speed X component</li>
  <li>ARIEL speed Y component</li>
  <li>ARIEL speed Z component</li>
  <li>ARIEL acceleration magnitude</li>
  <li>ARIEL acceleration X component</li>
  <li>ARIEL acceleration Y component</li>
  <li>ARIEL acceleration Z component</li>
  <li>ARIEL jerk magnitude</li>
  <li>ARIEL jerk X component</li>
  <li>ARIEL jerk Y component</li>
  <li>ARIEL jerk Z component</li>
  <li>STRN speed magnitude</li>
  <li>STRN speed X component</li>
  <li>STRN speed Y component</li>
  <li>STRN speed Z component</li>
  <li>STRN acceleration magnitude</li>
  <li>STRN acceleration X component</li>
  <li>STRN acceleration Y component</li>
  <li>STRN accelerationZ component</li>
  <li>STRN jerk magnitude</li>
  <li>STRN jerk X component</li>
  <li>STRN jerk Y component</li>
  <li>STRN jerk Z component</li>
  <li>RHEL speed magnitude</li>
  <li>RHEL speed X component</li>
  <li>RHEL speed Y component</li>
  <li>RHEL speed Z component</li>
  <li>RHEL acceleration magnitude</li>
  <li>RHEL acceleration X component</li>
  <li>RHEL acceleration Y component</li>
  <li>RHEL acceleration Z component</li>
  <li>RHEL jerk magnitude</li>
  <li>RHEL jerk X component</li>
  <li>RHEL jerk Y component</li>
  <li>RHEL jerk Z component</li>
  <li>LHEL speed magnitude</li>
  <li>LHEL speed X component</li>
  <li>LHEL speed Y component</li>
  <li>LHEL speed Z component</li>
  <li>LHEL acceleration magnitude</li>
  <li>LHEL acceleration X component</li>
  <li>LHEL acceleration Y component</li>
  <li>LHEL acceleration Z component</li>
  <li>LHEL jerk magnitude</li>
  <li>LHEL jerk X component</li>
  <li>LHEL jerk Y component</li>
  <li>LHEL jerk Z component</li>
  <li>RPLM speed magnitude</li>
  <li>RPLM speed X component</li>
  <li>RPLM speed Y component</li>
  <li>RPLM speed Z component</li>
  <li>RPLM acceleration magnitude</li>
  <li>RPLM acceleration X component</li>
  <li>RPLM acceleration Y component</li>
  <li>RPLM acceleration Z component</li>
  <li>RPLM jerk magnitude</li>
  <li>RPLM jerk X component</li>
  <li>RPLM jerk Y component</li>
  <li>RPLM jerk Z component</li>
  <li>LPLM speed magnitude</li>
  <li>LPLM speed X component</li>
  <li>LPLM speed Y component</li>
  <li>LPLM speed Z component</li>
  <li>LPLM acceleration magnitude</li>
  <li>LPLM acceleration X component</li>
  <li>LPLM acceleration Y component</li>
  <li>LPLM acceleration Z component</li>
  <li>LPLM jerk magnitude</li>
  <li>LPLM jerk X component</li>
  <li>LPLM jerk Y component</li>
  <li>LPLM jerk Z component</li>




</ol>

In [ ]:
# Questi sono tutte le features estratte
# kineticglobal,kineticchest,directnesshead,density,leftwristke,rightwristke,leftankleke,rightankleke,headke,posturaltension

# Queste sono le feature che utilizzeremo
# kineticglobal, density, leftwirstke, rightwristke, leftankleke, rightankleke, headke, posturaltension
# che corrispondono
# 2,4,5,6,7,8,9,10,11
# Questa funzione legge il file di input e restituisce un dataframe con i dati preprocessati
# ATTENZIONE
# il motivo per cui 4 corrisponde a density, mentre in realtà è il 3 è perchè df.drop prima non cancellava
# perciò il iloc[:,0] corrispondeva alla colonna time che era rimasta
# iloc considera le colonne come array
def ReadAndPreProcess(inputDataRaw):
    # lettura
    df=pd.read_csv(inputDataRaw,sep=' ', header=None)
    df=df.drop(0, axis=1)
    # i dati usciti da eyesweb hanno le feature inverse, qui le faccio rimettere nel ordine giusto
    df = df.iloc[:, ::-1]
    # non ha utilità di calcolo, serve solo come placeholder per il risultato col grafico
    #df["som"]=df.sum(axis=1)
    ## preprocessing

    # dealing NaN values
    #-serafino ha usato forward fill, backward fill, linear interpolation
    #-ricordo che serafino aveva gia utilizzato sta cosa sui dati grezzi non sulle feature ma sui dati prefeature percio dovrebbe essere gia apposto

    # downsampling
    #-sono a 100ms, non sò se devo scendere a 50ms. da decidere
    #-non lo faccio xk non mi interessa se va piu lento, guarda su notion per ulteriori informazioni

    # low pass filter
    #-Skogstad and colleagues (2013) e https://stackoverflow.com/questions/25191620/creating-lowpass-filter-in-scipy-understanding-methods-and-units
    #-implementare dopo

    # remove outliers
    #-utilizzare hampel filter

    # stretch
    #-forse devo stretcharlo come ho fatto precedentemente
    #-anche se nel codice precedente ho stretchato solo il groundtruth

    # ritorno un oggetto dataframe dopo che è stato lavorato, ottenendo un prodotto lavorato
    return df

In [4]:
# questa funzione ritorna un dataframe del groundtruth che viene usato specificatamente per visualizzare il gt
# è soggetto a un preprocessing dei dati siccome l'ultimo groundtruth è dove termina il ts del gt
# di conseguenza per farlo corrispondere, bisogna stretcharlo
# ma ricordo di aver rifatti i dati nuovi per generare un groundtruth a fine ts, da controllare cosi che non serve stretcharlo?
def LoadingGroundTruth(df,gtraw):
    gt=pd.read_csv(gtraw,sep=' ', header=None)
    gt=gt.iloc[:,0].values
    #stretching dei dati se necessario per farlo corrispondere alla ts dei dati
    stretch_gt = np.array([])
    for idx,i in enumerate(gt):
        relpos = len(df)*i/gt[-1]
        stretch_gt = np.append(stretch_gt,relpos)

    # eliminiamo l'ultimo elemento che è stato annotato solo per delimitare la lunghezza della gt simile alla ts
    return stretch_gt[:-1]

In [ ]:
# utilizzo CLASP 
# prende come parametro un dataframe e restituisce il clasp score
# gt e known vengono usati per usare il numero vero di cp se uguale a 1 sennò si cerca di predirlo se il modello lo permette
def GetClasp(df,gt,known, **kwargs):
    
    result=np.array([])
    eachresult = []
    eachclasp=[]
    for idx,i in enumerate(range(0,len(features_name))):
      
        ts=df.iloc[:,i]
        clasp = BinaryClaSPSegmentation(**kwargs)
        found_cps = clasp.fit_predict(ts.values)    

        # c'è un bug con binseg dove un cp è oltre la lunghezza del ts
        # faccio un loop e se eccede cambio il valore con la len(tf)-1
        for i in range(0,len(found_cps)):
            if found_cps[i] >= len(ts):
                found_cps[i] = len(ts)-1

        # per ogni array di cp di ogni singola feature
        # li unisco in un unico array. in pratica faccio un OR di tutti i cp
        result = np.sort(np.append(result,found_cps).flatten())
        result = np.unique(result)
        eachresult.append(found_cps)
        eachclasp.append(clasp)
        
        

        
    return result, eachresult, eachclasp
        
    


In [ ]:
# utilizzo CLASP 
# prende come parametro un dataframe e restituisce il clasp score
# gt e known vengono usati per usare il numero vero di cp se uguale a 1 sennò si cerca di predirlo se il modello lo permette
def GetClaspOld(df,gt,known, **kwargs):
    
    result=np.array([])
    eachresult = []
    eachclasp=[]
    for idx,i in enumerate([1,3,4,5,6,7,8]):
        
        ts=df.iloc[:,i]
        clasp = BinaryClaSPSegmentation(**kwargs)
        found_cps = clasp.fit_predict(ts.values)    

        # c'è un bug con binseg dove un cp è oltre la lunghezza del ts
        # faccio un loop e se eccede cambio il valore con la len(tf)-1
        for i in range(0,len(found_cps)):
            if found_cps[i] >= len(ts):
                found_cps[i] = len(ts)-1

        # per ogni array di cp di ogni singola feature
        # li unisco in un unico array. in pratica faccio un OR di tutti i cp
        result = np.sort(np.append(result,found_cps).flatten())
        result = np.unique(result)
        eachresult.append(found_cps)
        eachclasp.append(clasp)
        
        
        
    return result, eachresult, eachclasp

In [6]:
# utilizzo BINSEG
def GetBinseg(df,gt,known):
    result=np.array([])
    eachresult=[]
    for idx, i in enumerate([2,4,5,6,7,8,9,10,11]):
        
        ts=df.iloc[:,i]
        binseg = rpt.Binseg(model="ar").fit(ts.values)

        if known == 1:
            found_cps = np.array(binseg.predict(n_bkps=len(gt)), dtype=np.int64)
        else:
            found_cps = np.array(binseg.predict(pen=np.log(len(ts.values))), dtype=np.int64)
        
        # c'è un bug con binseg dove un cp è oltre la lunghezza del ts
        # faccio un loop e se eccede cambio il valore con la len(tf)-1
        for i in range(0,len(found_cps)):
            if found_cps[i] >= len(ts):
                found_cps[i] = len(ts)-1

        result = np.sort(np.append(result,found_cps).flatten())
        result = np.unique(result)
        eachresult.append(found_cps)

        

    return result, eachresult

In [7]:
#utilizzo WINDOW
def GetWindowSlide(df,gt,known):
    result=np.array([])
    eachresult=[]
    for idx, i in enumerate([2,4,5,6,7,8,9,10,11]):
        
        ts=df.iloc[:,i]
        binseg = rpt.Window(model="ar").fit(ts.values)

        if known==1:
            found_cps = np.array(binseg.predict(n_bkps=len(gt)), dtype=np.int64)
        else:
            found_cps = np.array(binseg.predict(pen=np.log(len(ts.values))), dtype=np.int64)
        
        # c'è un bug con binseg dove un cp è oltre la lunghezza del ts
        # faccio un loop e se eccede cambio il valore con la len(tf)-1
        for i in range(0,len(found_cps)):
            if found_cps[i] >= len(ts):
                found_cps[i] = len(ts)-1

        result = np.sort(np.append(result,found_cps).flatten())
        result = np.unique(result)
        eachresult.append(found_cps)

       

    return result, eachresult

In [8]:
# utilizzo FLUSS
def GetFluss(df,gt,known):
    result=np.array([])
    eachresult=[]
    for idx, i in enumerate([2,4,5,6,7,8,9,10,11]):

        ts=df.iloc[:,i]
        window_size = min(find_dominant_window_sizes(ts.values) * 2, len(ts.values) // 2)
        mp = stumpy.stump(ts.values, m=window_size)

        if known == 1:
            cac, found_cps = stumpy.fluss(mp[:, 1], L=window_size, n_regimes=len(gt))
        else:
            cac, found_cps = stumpy.fluss(mp[:, 1], L=window_size, n_regimes=4)
        
        # c'è un bug con binseg dove un cp è oltre la lunghezza del ts
        # faccio un loop e se eccede cambio il valore con la len(tf)-1
        for i in range(0,len(found_cps)):
            if found_cps[i] >= len(ts):
                found_cps[i] = len(ts)-1

        result = np.sort(np.append(result,found_cps).flatten())
        result = np.unique(result)
        eachresult.append(found_cps)

       

    return result, eachresult

In [9]:
# utilizzo GSS
def GetGSS(df,gt,known):
    result=np.array([])
    eachresult=[]
    for idx,i in enumerate([2,4,5,6,7,8,9,10,11]):

        ts=df.iloc[:,i]
        
        if known==1:
            ggs = GreedyGaussianSegmenter(k_max=len(gt))
        else:
            ggs = GreedyGaussianSegmenter(k_max=1)


        ggs.fit_predict(ts.values)
        found_cps = ggs.change_points_

        if (len(found_cps) >= 2):
            # remove start and end of the series
            found_cps = found_cps[1:-1] 

        # c'è un bug con binseg dove un cp è oltre la lunghezza del ts
        # faccio un loop e se eccede cambio il valore con la len(tf)-1
        for i in range(0,len(found_cps)):
            if found_cps[i] >= len(ts):
                found_cps[i] = len(ts)-1

        result = np.sort(np.append(result,found_cps).flatten())
        result = np.unique(result)
        eachresult.append(found_cps)

  
        
    return result,eachresult

In [10]:
# utilizzo IGS
def GetIGS(df,gt,known):
    result=np.array([])
    eachresult=[]
    for idx,i in enumerate([2,4,5,6,7,8,9,10,11]):

        ts=df.iloc[:,i]

        if known==1:
            igts = InformationGainSegmenter(k_max=len(gt))
        else:
            igts = InformationGainSegmenter(k_max=1)

        igts.fit_predict(ts.values)
        found_cps = igts.change_points_

        if (len(found_cps) >= 2):
            # remove start and end of the series
            found_cps = found_cps[1:-1] 

        # c'è un bug con binseg dove un cp è oltre la lunghezza del ts
        # faccio un loop e se eccede cambio il valore con la len(tf)-1
        for i in range(0,len(found_cps)):
            if found_cps[i] >= len(ts):
                found_cps[i] = len(ts)-1

        result = np.sort(np.append(result,found_cps).flatten())
        result = np.unique(result)
        eachresult.append(found_cps)


    return result, eachresult
            

In [11]:
#utiizzo STRAY
def GetSTRAY(df,gt,known):
    result=np.array([])
    eachresult=[]
    for idx,i in enumerate([2,4,5,6,7,8,9,10,11]):

        ts=df.iloc[:,i]
        stray = STRAY()
        stray.fit(ts.values)

        if known==1:
            found_cps = np.sort(np.argpartition(-stray.score_, len(gt))[:len(gt)])
        else:
            found_cps = np.sort(np.argpartition(-stray.score_, 1)[:1])

        if (len(found_cps) >= 2):
            # remove start and end of the series
            found_cps = found_cps[1:-1] 

        # c'è un bug con binseg dove un cp è oltre la lunghezza del ts
        # faccio un loop e se eccede cambio il valore con la len(tf)-1
        for i in range(0,len(found_cps)):
            if found_cps[i] >= len(ts):
                found_cps[i] = len(ts)-1

        result = np.sort(np.append(result,found_cps).flatten())
        result = np.unique(result)
        eachresult.append(found_cps)



    return result, eachresult


In [12]:
def fakeplot(df,gt,cp):
    _ = plot_series_with_change_points(df.iloc[:,7], cp[4], title="prediction number: "+str(7))

    plt.figure(figsize=(18,9))
    plt.plot(np.arange(len(df[7].values)),df[7].values,'blue',linewidth=0.5)
    for idx2,j in enumerate(gt.astype(int)):

        plt.axvline(x = j, color = 'green',linewidth=1) 

In [13]:
def PlotResult(df,gt,cp,eachcp,ts):
    #da testare quando ho piu valori
    #clasp.plot(gt_cps=gt.astype(int), heading="Segmentation of different umpire cricket signals", ts_name="ACC", file_path="segmentation_example.png")

        
    for idx,i in enumerate([2,4,5,6,7,8,9,10,11]):
  
        # idx == 8 è l'ultimo elemento. percio quando i è uguale a 11
        # significa che l'ultimo elemento del mio for è la ts totale di tutte le feature
        if idx == 8:
            plt.figure(figsize=(18,9))
            plt.plot(np.arange(len(df["som"].values)),df["som"].values,'blue',linewidth=0.5)
            for idx2,j in enumerate(gt.astype(int)):

                plt.axvline(x = j, color = 'green',linewidth=1) 

            for j in cp.tolist():
                plt.axvline(x = j, color = 'black',linewidth=1) 
        else:


            print("asd"+str(i))
            _ = plot_series_with_change_points(df.iloc[:,i], eachcp[idx], title=str(ts)+"prediction number: "+str(i))
            for idx2,j in enumerate(gt.astype(int)):
                plt.axvline(x = j, color = 'green',linewidth=1) 

   


In [14]:
# calcola i vari scores dati il groundtruth e il prediction
# puo salvare il risultato su file per evitare di perderli
# prende come parametro nome del groundtruth, groundtruth, nome della timeseries e il prediction
def Evaluate(modelName,gtName, gt, tsName, cp, df):
    # creo dei array di lunghezza come la ts cosi possono fare il confronto
    # sia per il gt che per il pd
  
    cpnump = np.array(cp)
    gtnump = np.array(gt)

    cp_long = np.zeros(len(df)+1)
    cp_long[cpnump.astype(int)]=1

    gt_long = np.zeros(len(df)+1)
    gt_long[gtnump.astype(int)]=1

    accuracy = accuracy_score(gt_long, cp_long)
    precision = precision_score(gt_long,cp_long)
    recall = recall_score(gt_long,cp_long)
    f1 = f1_score(gt_long,cp_long)
    print(f'accuracy, precision, recall, f1: {accuracy,precision,recall,f1}')
    
    #scrivo su file il risultato
    f = open("outputFile/"+str(modelName), "a")
    f.write("model:"+str(tsName)+"\n")
    f.write("Timeseries:"+str(tsName)+" accuracy:"+str(accuracy)+" precision:"+str(precision)+" recall:"+str(recall)+" f1:"+str(f1)+" \n")
    f.write("\n")
    f.close()

In [ ]:
def f1scoremargin(ground_truth, predictions, tolerance):
    """
    Calcola l'F1 score con una finestra di tolleranza sui change points.
    
    :param ground_truth: Lista o array di change points reali
    :param predictions: Lista o array di change points predetti
    :param tolerance: La tolleranza temporale (numero di unità temporali)
    :return: precision, recall, f1-score
    """
    ground_truth = np.array(ground_truth)
    predictions = np.array(predictions)
    
    # Vettori per tracciare quali punti sono stati già associati
    matched_ground_truth = np.zeros(len(ground_truth), dtype=bool)
    matched_predictions = np.zeros(len(predictions), dtype=bool)

    mgt={key: False for key in ground_truth}
    mcp={key: False for key in predictions}
    print(f'gt:{len(ground_truth)} - cp:{len(predictions)}')
    # True Positives (TP)
    tp = 0
    for i, gt_point in enumerate(ground_truth):
        for j, pred_point in enumerate(predictions):
            
            if not matched_predictions[j] and abs(gt_point - pred_point) <= tolerance:
                tp += 1
                matched_ground_truth[i] = True
                matched_predictions[j] = True

                mgt[gt_point] = True
                mcp[pred_point] = True
                break
            
    
    # False Positives (FP) - predizioni non corrispondenti a nessun ground truth entro la tolleranza
    fp = np.sum(~matched_predictions)
    
    # False Negatives (FN) - punti del ground truth non corrispondenti a nessuna predizione entro la tolleranza
    fn = np.sum(~matched_ground_truth)
    #print(f'tp:{tp} - fp:{fp} - fn:{fn}')
    #print(mgt)
    #print(mcp)
    # Calcolo di precision, recall e F1-score
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1

In [ ]:
def IgnoreZone(idx,cpraw):
    cp = cpraw.tolist()
    if idx == 0:
        for i in range(len(cp)-1, -1, -1):
            if cp[i] > 3944.7118557910376+100 and cp[i] < 5911.693516853054-100 or cp[i] > 12845.0+100:
                cp.pop(i)
                
        
    elif idx == 1:
        for i in range(len(cp)-1, -1, -1):
            if cp[i]< 969.6180827886711-100 and cp[i] > 13125.469063180826+100:
                cp.pop(i)
    elif idx == 2:
        for i in range(len(cp)-1, -1, -1):
            if cp[i] > 2874.607407407407+100 and cp[i] < 4016.935849056604-100:
                cp.pop(i)

    elif idx == 17:
        for i in range(len(cp)-1, -1, -1):
            if cp[i] > 180.03455207940698+100 and cp[i] < 1227.051137077522-100 or cp[i] > 5865.505591154668+100:
                cp.pop(i)

    elif idx == 18:
        for i in range(len(cp)-1, -1, -1):
            if cp[i] > 138.33224102186853+100 and cp[i] < 3677.231833076974-100:
                cp.pop(i)


    elif idx == 20:
        for i in range(len(cp)-1, -1, -1):
            if cp[i] > 8187.634803581529+100:
                cp.pop(i)


    else:
        print("error IgnoreZone()")
    return np.array(cp)

In [ ]:
def delnear(arr,range):
    i = 0
    while i < len(arr) - 1:
        # Iniziamo con il primo elemento di un potenziale gruppo
        gruppo_inizio = i
        gruppo_fine = i

        # Cerca gli elementi che fanno parte dello stesso gruppo
        while gruppo_fine < len(arr) - 1 and arr[gruppo_fine + 1] - arr[gruppo_fine] < range:
            gruppo_fine += 1

        # Se esiste un gruppo di più elementi
        if gruppo_fine > gruppo_inizio:
            # Se la distanza tra l'inizio e la fine è minore di 50, elimina l'elemento maggiore (gruppo_fine)
            if arr[gruppo_fine] - arr[gruppo_inizio] < range:
                arr = np.delete(arr, gruppo_fine)
            
            # Elimina tutti gli elementi interni al gruppo
            arr = np.concatenate((arr[:gruppo_inizio + 1], arr[gruppo_fine:]))

        # Procedi con il prossimo gruppo
        i = gruppo_inizio + 1

    return arr

In [ ]:
timeseries=[
      "in\cora1_in.txt",
      "in\cora4_05_in.txt",
      "in\cora4_08_in.txt",
      "in\cora5_in.txt",
      "in\cora14_in.txt",
      "in\marianne7_in.txt",
      "in\marianne8_in.txt",
      "in\marianne10_in.txt",
      "in\marianne18_in.txt",
      "in\marianne19_in.txt",
      "in\marianne24_in.txt",
      "in\marianne26_in.txt",
      "in\marianne41_in.txt",
      "in\marianne42_in.txt",
      "in\marianne43_in.txt",
      "in\marianne47_in.txt",
      "in\marianne48_in.txt",
      "in\muriel18_in.txt",
      "in\muriel26_in.txt",
      "in\muriel27_in.txt",
      "in\muriel30_in.txt"
      ]
groundtruth=[
      "gt\cora_gt_2019-08-08_t001_video01.txt",
      "gt\cora_gt_2019-05-22_t004_video01.txt",
      "gt\cora_gt_2019-08-08_t004_video01.txt",
      "gt\cora5_gt.txt",
      "gt\cora_gt_2019-08-08_t014_video01.txt",
      "gt\marianne_gt_2016-03-22_t007_video01.txt",
      "gt\marianne_gt_2016-03-22_t008_video01.txt",
      "gt\marianne_gt_2016-03-22_t010_video01.txt",
      "gt\marianne_gt_2016-03-22_t018_video01.txt",
      "gt\marianne_gt_2016-03-22_t019_video01.txt",
      "gt\marianne_gt_2016-03-22_t024_video01.txt",
      "gt\marianne_gt_2016-03-22_t026_video01.txt",
      "gt\marianne_gt_2016-03-22_t041_video01.txt",
      "gt\marianne_gt_2016-03-22_t042_video01.txt",
      "gt\marianne_gt_2016-03-22_t043_video01.txt",
      "gt\marianne_gt_2016-03-22_t047_video01.txt",
      "gt\marianne_gt_2016-03-22_t048_video01.txt",
      "gt\muriel_gt_2016-03-21_t018_video01.txt",
      "gt\muriel_gt_2016-03-21_t026_video01.txt",
      "gt\muriel_gt_2016-03-21_t027_video01.txt",
      "gt\muriel_gt_2016-03-23_t030_video01.txt"
      ]

# CLASP with known number of cp
for i in range(0,len(timeseries)):
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp,clasp=GetClasp(df,gt,1)
    PlotResult(df,gt,cp,eachcp,timeseries[i])
    Evaluate("CLASPk",groundtruth[i],gt,timeseries[i],cp,df)

In [16]:
# CLASP without knowing cp
for i in range(0,1):#len(timeseries)):
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp,clasp=GetClasp(df,gt,0,n_jobs=8,window_size="fft", distance="euclidean_distance", excl_radius=4)
    PlotResult(df,gt,cp,eachcp,timeseries[i])
    Evaluate("CLASP",groundtruth[i],gt,timeseries[i],cp,df)

TypeError: GetClasp() got an unexpected keyword argument 'n_jobs'

# BINSEG with known number of cp
for i in range(0,len(timeseries)):
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp=GetBinseg(df,gt,1)
    PlotResult(df,gt,cp,eachcp,timeseries[i])
    Evaluate("BINSEGw",groundtruth[i],gt,timeseries[i],cp,df)

# BINSEG without knowing cp
for i in range(0,len(timeseries)):
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp=GetBinseg(df,gt,0)
    PlotResult(df,gt,cp,eachcp,timeseries[i])
    Evaluate("BINSEG",groundtruth[i],gt,timeseries[i],cp,df)

# WINDOWSLIDE with known number of cp
for i in range(0,len(timeseries)):
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp=GetWindowSlide(df,gt,1)
    PlotResult(df,gt,cp,eachcp,timeseries[i])
    Evaluate("WINDOWw",groundtruth[i],gt,timeseries[i],cp,df)

# WINDOWSLIDE without knowing cp
for i in range(0,len(timeseries)):
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp=GetWindowSlide(df,gt,0)
    PlotResult(df,gt,cp,eachcp,timeseries[i])
    Evaluate("WINDOW",groundtruth[i],gt,timeseries[i],cp,df)

# FLUSS with known number of cp
for i in range(0,len(timeseries)):
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp=GetFluss(df,gt,1)
    PlotResult(df,gt,cp,eachcp,timeseries[i])
    Evaluate("FLUSSw",groundtruth[i],gt,timeseries[i],cp,df)

# FLUSS without knowing cp
for i in range(0,len(timeseries)):
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp=GetFluss(df,gt,0)
    PlotResult(df,gt,cp,eachcp,timeseries[i])
    Evaluate("FLUSS",groundtruth[i],gt,timeseries[i],cp,df)

I MODELLI SOTTO NO XK SERVONO DATI MULTIVARIABILI E I MIEI SONO SINGOLI

# GSS with known number of cp
for i in range(0,len(timeseries)):
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp=GetGSS(df,gt,1)
    PlotResult(df,gt,cp,eachcp,timeseries[i])
    Evaluate("GSSw",groundtruth[i],gt,timeseries[i],cp,df)

# GSS without knowing cp
for i in range(0,len(timeseries)):
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp=GetGSS(df,gt,0)
    PlotResult(df,gt,cp,eachcp,timeseries[i])
    Evaluate("GSS",groundtruth[i],gt,timeseries[i],cp,df)

# IGS with known number of cp
for i in range(0,len(timeseries)):
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp=GetIGS(df,gt,1)
    PlotResult(df,gt,cp,eachcp,timeseries[i])
    Evaluate("IGSw",groundtruth[i],gt,timeseries[i],cp,df)

# IGS without knowing cp
for i in range(0,len(timeseries)):
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp=GetIGS(df,gt,0)
    PlotResult(df,gt,cp,eachcp,timeseries[i])
    Evaluate("IGS",groundtruth[i],gt,timeseries[i],cp,df)